In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

In [2]:
#load json data
data_path = os.path.join('Resources','purchase_data.json')
data_path

'Resources\\purchase_data.json'

In [3]:
#open json file and print json data to find keys and understand data
with open (data_path) as game_json:
    game_data = json.load(game_json)
    print (json.dumps(game_data, indent = 4, sort_keys = True))

[
    {
        "Age": 38,
        "Gender": "Male",
        "Item ID": 165,
        "Item Name": "Bone Crushing Silver Skewer",
        "Price": 3.37,
        "SN": "Aelalis34"
    },
    {
        "Age": 21,
        "Gender": "Male",
        "Item ID": 119,
        "Item Name": "Stormbringer, Dark Blade of Ending Misery",
        "Price": 2.32,
        "SN": "Eolo46"
    },
    {
        "Age": 34,
        "Gender": "Male",
        "Item ID": 174,
        "Item Name": "Primitive Blade",
        "Price": 2.46,
        "SN": "Assastnya25"
    },
    {
        "Age": 21,
        "Gender": "Male",
        "Item ID": 92,
        "Item Name": "Final Critic",
        "Price": 1.36,
        "SN": "Pheusrical25"
    },
    {
        "Age": 23,
        "Gender": "Male",
        "Item ID": 63,
        "Item Name": "Stormfury Mace",
        "Price": 1.27,
        "SN": "Aela59"
    },
    {
        "Age": 20,
        "Gender": "Male",
        "Item ID": 10,
        "Item Name": "Sleepwalker",
  

In [4]:
#convert data into panda dataframe
game_pd = pd.DataFrame(game_data)
game_pd.head(3)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25


In [5]:
#finding number of rows, which is equivalent to number of players
numbers = game_pd['SN'].nunique()
num_players = {'Total Players':[numbers]}
num_pd = pd.DataFrame.from_dict(num_players)
num_pd

,Total Players
0,573


In [6]:
#run calculations
num_items = len(game_pd['Item ID'].unique())
price_mean = game_pd['Price'].mean()
purchase_count = game_pd['Price'].count()
total_purchase = game_pd['Price'].sum()

total_analysis = {'Number of Unique Items':[num_items], 'Average Purchase Price':[price_mean], 'Total Number of Purchases':[purchase_count], 'Total Revenue':[total_purchase]}
total_analysis_pd = pd.DataFrame.from_dict(total_analysis)
total_analysis_pd = total_analysis_pd[['Number of Unique Items','Average Purchase Price', 'Total Number of Purchases','Total Revenue']]

#data formatting
total_analysis_pd = total_analysis_pd.round(2)
total_analysis_pd['Average Purchase Price'] = total_analysis_pd['Average Purchase Price'].map("${:,.2f}".format)
total_analysis_pd['Total Revenue'] = total_analysis_pd['Total Revenue'].map("${:,.2f}".format)
total_analysis_pd

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [7]:
#create series by gender, sort and then grouped
gender_pd = game_pd['Gender']
gender_sorted = gender_pd.sort_values()
gender_grouped = gender_sorted.value_counts()

percent = []
gender_demo = gender_grouped.to_frame(name = 'Count')

#calculate percentage of gender demographics
for x in gender_demo['Count']:
    percent.append((x/game_pd.shape[0])*100)

#add another column to the current dataframe to include percentages
gender_demo['Percent'] = percent
gender_demo = gender_demo.round(2)
gender_demo['Percent'] = gender_demo['Percent'].map('{:,.2f}%'.format)
gender_demo

,Count,Percent
Male,633,81.15%
Female,136,17.44%
Other / Non-Disclosed,11,1.41%


In [8]:
#Purchasing Analysis by gender
purchase_gender = game_pd.loc[:,['Gender','Price']]
purchase_count = purchase_gender.groupby(['Gender']).count()['Price'].rename('Purchase Count')

purchase_avgerage= purchase_gender.groupby(['Gender']).mean()['Price'].rename('Average Purchase Price')

purchase_sum = purchase_gender.groupby(['Gender']).sum()['Price'].rename('Total Purchase Value')

purchase_norm = purchase_sum/gender_demo['Count']
purchase_norm = pd.DataFrame({"Normalized Totals":purchase_norm})

#dataframe formatting
purchase_total = pd.concat([purchase_count, purchase_avgerage, purchase_sum, purchase_norm], axis = 1)
purchase_total = purchase_total.round(2)
purchase_total['Average Purchase Price'] = purchase_total['Average Purchase Price'].map("${:,.2f}".format)
purchase_total['Total Purchase Value'] = purchase_total['Total Purchase Value'].map("${:,.2f}".format)
purchase_total['Normalized Totals'] = purchase_total['Normalized Totals'].map("${:,.2f}".format)

#display table
purchase_total

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$2.82
Male,633,$2.95,"$1,867.68",$2.95
Other / Non-Disclosed,11,$3.25,$35.74,$3.25


In [9]:
#establish age groups
age_group = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Categorize the existing players using the age bins
game_pd["Age Ranges"] = pd.cut(game_pd["Age"], age_group, labels=group_names)

#calculations
age_demo_totals = game_pd['Age Ranges'].value_counts()
age_demo_percents = age_demo_totals/numbers * 100
age_demo_percents = age_demo_percents.map('{:,.2f}%'.format)
age_demo = pd.DataFrame({'Total Count':age_demo_totals, 'Percentage of Players':age_demo_percents} )
age_demo = age_demo.round(2)

age_demo.sort_index()

,Percentage of Players,Total Count
<10,4.89%,28
10-14,6.11%,35
15-19,23.21%,133
20-24,58.64%,336
25-29,21.82%,125
30-34,11.17%,64
35-39,7.33%,42
40+,2.97%,17


In [10]:
game_pd.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Ranges
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [11]:
#calculations for age purchase summary
age_purchase_count = game_pd.groupby(['Age Ranges']).count()['Price'].rename('Purchase Count')
age_purchase_average = game_pd.groupby(['Age Ranges']).mean()['Price'].rename('Average Purchase Price')
age_purchase_total = game_pd.groupby(['Age Ranges']).sum()['Price'].rename('Total Purchase Value')

age_purchase_normal = age_purchase_total/age_demo['Total Count']

# Convert to DataFrame
age_data = pd.DataFrame({"Purchase Count": age_purchase_count, "Average Purchase Price": age_purchase_average, "Total Purchase Value": age_purchase_total, "Normalized Totals": age_purchase_normal})

#data formatting
age_data = age_data.round(2)
age_data['Average Purchase Price'] = age_data['Average Purchase Price'].map('${:,.2f}'.format)
age_data['Normalized Totals'] = age_data['Normalized Totals'].map('${:,.2f}'.format)
age_data['Total Purchase Value'] = age_data['Total Purchase Value'].map('${:,.2f}'.format)

age_data = age_data.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
age_data.sort_index()

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
10-14,35,$2.77,$96.95,$2.77
15-19,133,$2.91,$386.42,$2.91
20-24,336,$2.91,$978.77,$2.91
25-29,125,$2.96,$370.33,$2.96
30-34,64,$3.08,$197.25,$3.08
35-39,42,$2.84,$119.40,$2.84
40+,17,$3.16,$53.75,$3.16
<10,28,$2.98,$83.46,$2.98


In [12]:
#run calculations
user_total = game_pd.groupby(["SN"]).sum()["Price"].rename("Total Purchase Price")
user_average = game_pd.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = game_pd.groupby(["SN"]).count()["Price"].rename("Purchase Count")

#convert to dataframe
user_data = pd.DataFrame({'Total Purchase Price':user_total, 'Average Purchase Price':user_average,'Purchase Count':user_count})

#sort the data by largest purchase value
user_sorted = user_data.sort_values("Total Purchase Price", ascending = False)

#data format
user_sorted['Total Purchase Price'] = user_sorted['Total Purchase Price'].map('${:,.2f}'.format)
user_sorted['Average Purchase Price'] = user_sorted['Average Purchase Price'].map('${:,.2f}'.format)

user_sorted = user_sorted.loc[:,['Purchase Count','Total Purchase Price','Average Purchase Price']]
top_spenders = user_sorted.head(5)
top_spenders

,Purchase Count,Total Purchase Price,Average Purchase Price
SN,,,
Undirrala66,5,$17.06,$3.41
Saedue76,4,$13.56,$3.39
Mindimnya67,4,$12.74,$3.18
Haellysu29,3,$12.73,$4.24
Eoda93,3,$11.58,$3.86


In [13]:
#extract item data
item_data = game_pd.loc[:,['Item ID','Item Name','Price']]

#calculations
average_item_price = item_data.groupby(['Item ID','Item Name']).mean()['Price'].rename('Average Item Price')
item_count = item_data.groupby(['Item ID','Item Name']).count()['Price'].rename('Purchase Count')
total_item_purchase = item_data.groupby(['Item ID','Item Name']).sum()['Price'].rename('Total Purchase Value')

item_data_pd = pd.DataFrame({'Purchase Count':item_count,'Average Item Price':average_item_price,'Total Purchase Value':total_item_purchase})

popular_item = item_data_pd.sort_values(by = 'Purchase Count', ascending = False)
popular_item['Average Item Price'] = popular_item['Average Item Price'].map('${:,.2f}'.format)
popular_item['Total Purchase Value'] = popular_item['Total Purchase Value'].map('${:,.2f}'.format)
popular_item.head(5)

,,Average Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
84,Arcane Gem,$2.23,11,$24.53
31,Trickster,$2.07,9,$18.63
175,Woeful Adamantite Claymore,$1.24,9,$11.16
13,Serenity,$1.49,9,$13.41


In [14]:
#calculations for most profitable item
item_profitable = item_data_pd.sort_values(by = 'Total Purchase Value', ascending = False)
item_profitable['Average Item Price'] = item_profitable['Average Item Price'].map('${:,.2f}'.format)
item_profitable['Total Purchase Value'] = item_profitable['Total Purchase Value'].map('${:,.2f}'.format)
item_profitable.head(5)

,,Average Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,$4.14,9,$37.26
115,Spectral Diamond Doomblade,$4.25,7,$29.75
32,Orenmir,$4.95,6,$29.70
103,Singed Scalpel,$4.87,6,$29.22
107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
